# Программирование на Python
## Семинар 3. Повторение основ
#### Задача с прошлого занятия
Мы хотим автоматизировать процесс перекладки текстовых файлов с данными из хранилища в БД (Postgres). Предположим, что таблицы каждый раз разные. Соответственно, перед записью данных необходимо создать таблицу. Это можно сделать примерно так:

```
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);
```

PostgreSQL [располагает](https://www.postgresql.org/docs/current/datatype.html) следующими типами данных (это не все, что есть, но все, что нам сейчас пригодится):

<table>
    <tr>
        <th>Python</th>
        <th>PostgreSQL</th>
    </tr>
    <tr>
        <td>int</td>
        <td>smallint, integer, bigint</td>
    </tr>
    <tr>
        <td>float</td>
        <td>real</td>
    </tr>
    <tr>
        <td>bool</td>
        <td>boolean</td>
    </tr>
    <tr>
        <td>str</td>
        <td>varchar, text</td>
    </tr>
</table>

Реализуйте соответствующую функцию. Она должна иметь следующие аргументы:

- `filename` (строка) - путь к текстовому файлу;
- `sep` (строка, по дефолту для CSV) - разделитель;
- `tablename` (строка, по дефолту соответствует названию файла без расширения);
- `varnames` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`);
- `dtypes` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`).

Функция должна записывать получившийся скрипт в файл 'create_table.sql' и возвращать путь к этому файлу.

In [ ]:
# check if bool
# check if int -> smallint, integer, bigint
# check if float
# else text

In [2]:
big_list = list(range(100000))
big_set = set(range(100000))

In [3]:
%%timeit

_ = 50000 in big_list

722 µs ± 43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%%timeit

_ = 50000 in big_set

57.5 ns ± 2.41 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
-.5

-0.5

In [11]:
5.

5.0

In [12]:
all([True, False, True])

False

In [13]:
all([1, 0, 1])

False

In [23]:
all(i % 2 == 0 for i in range(int(1e150)))

False

In [67]:
import re

template = 'CREATE TABLE "{tablename}" (\n{ddl});'
trash_symbols = '" \n'
re_int = re.compile('-{0,1}\d+')
re_float = re.compile('-?\d+\.\d*')

def check_bool(value: str) -> bool:
    return value.lower() in {'true', 'false'}

def check_float(value: str) -> bool:
    return re_float.fullmatch(value) is not None

def check_int(value: str) -> bool:
    return re_int.fullmatch(value) is not None

def check_dtype_(vector, fun) -> bool:
    # all or nothing
    return all(fun(elem) for elem in vector)

def guess_dtype(vector) -> str:
    if check_dtype_(vector, check_bool):
        return 'bool'
    elif check_dtype_(vector, check_int):
        return 'int'
    elif check_dtype_(vector, check_float):
        return 'float'
    else:
        return 'text'

def extract_name(filename: str) -> str:
    dot_position = filename.rfind('.')
    slash_position = filename.rfind('/')

    return filename[(slash_position + 1):dot_position]

def create_sql(tname, varnames, dtypes) -> str:
    ddl = ''.join([f'    "{name}" {dtype},\n' for name, dtype in zip(varnames, dtypes)])

    return template.format(tablename=tname, ddl=ddl)

def sql_from_file(
    filename: str,
    sep=',',
    tablename=None,
    varnames=None,
    dtypes=None,
    file_out='sql_script.sql'
) -> str:
    # parse file
    with open(filename, 'r') as file:  # file = open('./StudentsPerformance.csv', 'r')
        lines = []
        
        for line in file:
            line_clean = [elem.strip(trash_symbols) for elem in line.split(sep)]
            
            lines.append(line_clean)

    # get filename
    if tablename is None:
        tablename = extract_name(filename)

    # get varnames
    if varnames is None:
        varnames = lines[0].copy()  # list(lines[0]), чтобы не создавать ссылку на первый список в lines

    # reshape lines
    lines_reshaped = list(zip(*lines[1:]))

    # guess dtypes
    if dtypes is None:
        dtypes = [guess_dtype(col) for col in lines_reshaped]  # get pairs (varname, dtype)

    # get sql
    sql = create_sql(tablename, varnames, dtypes)

    # write file
    with open(file_out, 'w') as file:
        file.write(sql)

    return file_out

In [68]:
path = '../Занятие 1/StudentsPerformance.csv'

name = sql_from_file(
    path
)

In [44]:
tname, vname, dts

('StudentsPerformance',
 ['gender',
  'race/ethnicity',
  'parental level of education',
  'lunch',
  'test preparation course',
  'math score',
  'reading score',
  'writing score'],
 ['text', 'text', 'text', 'text', 'text', 'int', 'int', 'int'])

In [65]:
template = 'CREATE TABLE "{tablename}" (\n{ddl});'

In [76]:
ddl = ''.join([f'    "{name}" {dtype},\n' for name, dtype in zip(vname, dts)])  #.rstrip(',')
ddl= ddl[:ddl.rfind(',')] + '\n'

In [77]:
print(template.format(tablename=tname, ddl=ddl))

CREATE TABLE "StudentsPerformance" (
    "gender" text,
    "race/ethnicity" text,
    "parental level of education" text,
    "lunch" text,
    "test preparation course" text,
    "math score" int,
    "reading score" int,
    "writing score" int
);


In [46]:
template

'\nCREATE TABLE distributors (\n    did integer,\n    name varchar(40)\n);\n'

In [18]:
check_dtype(c[-1], check_int)

True

In [5]:
re_int = re.compile('-{0,1}\d+')  # [0-9], -{0,1} ~ '-?'

In [8]:
re_int.match('-961')

In [9]:
re_int.fullmatch('-961')

<re.Match object; span=(0, 4), match='-961'>

In [57]:
re_float.match('-5.862')

<re.Match object; span=(0, 3), match='-5.'>

In [58]:
re_float.match('-5.862').group()

'-5.'

#### Задача 1

Известно, что подготовка и исследование данных для машинного обучения занимают куда больше времени, чем собственно машинное обучение. В частности, существует такая процедура, как создание т. н. dummy-переменных.

![dummy](https://www.statology.org/wp-content/uploads/2021/02/dummyvartrap1-768x344.png)

В `pandas` уже есть функция, которая принимает на вход текстовую колонку таблицы и возвращает вместо нее много колонок с dummy-переменными. Однако проблема в том, что в ваших данных есть переменные, значения которых представляют из себя словосочетания со знаками препинания (количество таких слов формально не ограничено). Некоторые алгоритмы "любят" только простые названия переменных вида `variable` или `simple_variable`. Кроме того, так или иначе для последующей работы неплохо было бы стандартизировать все названия.

Поэтому вам нужно создать функцию, которая бы принимала на вход список из значений переменной и производила следующую предобработку всех значений в колонке:

- удаление всех знаков препинания;
- приведение всех букв к нижнему регистру;
- замену всех пробелов на нижнее подчеркивание;
- ограничение длины каждого слова первыми четырьмя буквами.

Функция должна возвращать словарь (т. н. mapper), по которому можно преобразовать переменную в новый вид (с помощью методов `.map()` / `.apply` - о них вам расскажут позднее). Ключами должны быть старые значения, а собственно значениями - новые.

Не всегда бывает так, что значения переменной написаны на латинице. Напишите вашу функцию таким образом, чтобы по дефолту она обрабатывала англоязычные переменные, но также имела возможность (при соответствующем значении аргумента) обработать и кириллические. Вам может пригодиться функция `translit` из модуля `transliterate` (пример работы с кириллицей можете найти ниже).

**Пример**

```
function input:
['Agree', 'Agree', 'Neither agree nor disagree', "Don't know", 'Neither agree nor disagree', 'Neither agree nor disagree', 'Disagree', 'No answer', 'Agree strongly', 'No answer', 'Agree', 'Refusal', 'Refusal', 'Disagree strongly', 'Disagree']

function return:
{
    'Agree strongly': 'agre_stro',
    'Agree': 'agre',
    'Neither agree nor disagree': 'neit_agre_nor_disa',
    'Disagree': 'disa',
    'Disagree strongly': 'disa_stro',
    'Refusal': 'refu',
    "Don't know": 'dont_know',
    'No answer': 'no_answ'
}
```

In [ ]:
# пример работы функции translit

from transliterate import translit

print(translit('Полностью не согласен', 'ru', reversed=True))

In [ ]:
# наш код здесь

#### Задача 2
Предположим, что перед нами стоит задача создать dummy-переменные в PostgreSQL. К сожалению, это не `pandas`, и готовой функции на этот случай не предусмотрено. Однако вы знаете, что можно создавать новые переменные базируясь на значении старых используя конструкцию case...when...then...end:

```
PostgreSQL:
CASE WHEN <condition> THEN <value1> ELSE <value2> END AS <variable_name>
В ТОМ СЛУЧАЕ КОГДА <условие> ТОГДА <значение1> ИНАЧЕ <значение2> КОНЕЦ НАЗВАТЬ <имя_переменной>

Python:
if <condition>:
    <value1>
else:
    <value2>
```

Если категорий немного, то написать такой код несложно. Однако что делать, если их, к примеру, 100? В этом случае придется писать `CASE WHEN` столько раз, сколько уникальных значений содержит ваша переменная, да еще и придумывать каждый раз соответствующее название.

Используйте свои знания Python, чтобы автоматизировать процесс написания SQL-запроса. Напишите соответствующую функцию. Для автоматизированной генерации названия переменной используйте наработки предыдущей задачи. Учтите, что значения переменной не обязательно будут написаны на латинице.

На выходе функция должна печатать (в таком же формате, как в примере ниже) законченный блок SQL-запроса.

**Пример**
```
function return:
    CASE WHEN gincdif = 'Agree strongly' THEN 1 ELSE 0 END AS agre_stro,
    CASE WHEN gincdif = 'Agree' THEN 1 ELSE 0 END AS agre,
    CASE WHEN gincdif = 'Neither agree nor disagree' THEN 1 ELSE 0 END AS neit_agre_nor_disa,
    CASE WHEN gincdif = 'Disagree' THEN 1 ELSE 0 END AS disa,
    CASE WHEN gincdif = 'Disagree strongly' THEN 1 ELSE 0 END AS disa_stro,
    CASE WHEN gincdif = 'Refusal' THEN 1 ELSE 0 END AS refu,
    CASE WHEN ginsdif = "Don't know" THEN 1 ELSE 0 END AS dont_know,
    CASE WHEN ginsdif = "No answer" THEN 1 ELSE 0 END AS no_answ
```

In [ ]:
# наш код здесь